In [8]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time

## Подход

В qr-кодах существуют специальные паттерны для детекции. Они называются finder pattern, и представляют из себя кватраты, вложенные друг в друга с соотношением сторон: 3/5/7. Будем выделять фильтрами все границы на изображении, и если встретим контура, вложенные друг в друга с соотношением площадей 9/25/49, это и будет искомый finder-pattern qr-кода

![](finder-pattern.jpg "")

## Реализация

In [33]:
SIGMA = 7
THRESH_BLOCK_SIZE = 11
THRESH_C = 2
ERODE_ITERATIONS = 1
CANNY_THRESHHOLD_1 = 80
CANNY_THRESHHOLD_2 = 110
AREA_1 = 49
AREA_2 = 25
AREA_3 = 9

In [42]:
def image_preprocess(image):
    # черно-белый фильтр
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.GaussianBlur(image, (SIGMA, SIGMA), 0)
    image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, THRESH_BLOCK_SIZE, THRESH_C)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, np.ones((4, 4), np.uint8))
    image = cv2.erode(image, np.ones((3, 3)), iterations=ERODE_ITERATIONS)
    return image

def get_edges(image):
    # выделение границ
    edges = cv2.Canny(image, CANNY_THRESHHOLD_1, CANNY_THRESHHOLD_2)
    edges = (edges != 0).astype(np.uint8)
    edges = cv2.dilate(edges, np.ones((3, 3)))
    return edges

def prop_correct(areas, threshold_rate):
    # проверка площадей на правильное отношение
    outer_black_area = areas[0]
    for i, inner_white_area in enumerate(areas[1:-1]):
        for inner_black_area in areas[i+1:]:
            is_ratio_1 = np.abs(outer_black_area / inner_white_area - AREA_1 / AREA_2) < (AREA_1 / AREA_2) * threshold_rate
            is_ratio_2 = np.abs(outer_black_area / inner_black_area - AREA_1 / AREA_3) < (AREA_1 / AREA_3) * threshold_rate
            if is_ratio_1 and is_ratio_2:
                    return True
    return False

def get_pattern_contours(contours, hierarchy):
    # поиск подходящих контуров
    all_countours = set()
    pattern_contours = []
    for i, contour in enumerate(contours):
        if i in all_countours:
            continue
        areas = []
        processed_contours = set()
        j = i
        while hierarchy[j][2] != -1:
            areas.append(cv2.contourArea(contours[j]))
            processed_contours.add(j)
            j = hierarchy[j][2]
        if 3 < len(areas) < 7:
            if prop_correct(areas, 0.3):
                pattern_contours.append(contour)
                all_countours.union(processed_contours)
    return pattern_contours

def find_pattern(input_dir, output_dir):
    for image in os.listdir(input_dir):
        img = cv2.imread(os.path.join(input_dir, image))
        contours, hierarchy = cv2.findContours(get_edges(image_preprocess(img)), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        pattern_contours = get_pattern_contours(contours, hierarchy[0])
        cv2.drawContours(img, pattern_contours, -1, (0, 0, 255), 6)
        cv2.imwrite(os.path.join(output_dir, image), img)

In [43]:
find_pattern('TestSet1', 'Result1')

In [44]:
find_pattern('TestSet2', 'Result2')

In [48]:
input_dir = 'TestSet1'
output_dir = 'Result1'

start = time.time()
find_pattern(input_dir, output_dir)
end = time.time()

print(f"Всего времени на обработку {input_dir}: {end - start:.2f} секунд, время на одну картинку: {(end - start)/len(os.listdir(input_dir)):.2f} секунд")

Всего времени на обработку TestSet1: 9.50 секунд, время на одну картинку: 0.20 секунд


In [49]:
input_dir = 'TestSet2'
output_dir = 'Result2'

start = time.time()
find_pattern(input_dir, output_dir)
end = time.time()

print(f"Всего времени на обработку {input_dir}: {end - start:.2f} секунд, время на одну картинку: {(end - start)/len(os.listdir(input_dir)):.2f} секунд")

Всего времени на обработку TestSet2: 10.32 секунд, время на одну картинку: 0.22 секунд


## Результаты

Если считать, что выделение внешнего края внешнего квадрата - это ошибка

то есть, если алгоритм выделил нужный квадрат (один из трех finder pattern-ов) по врешнему контуру - это true positive, если он выделил по внутреннему контуру, это false positive, а если же он не выделил нужный квадрат, то это false negative

TestSet1:
- Precision: 0.71 
- Recall: 0.87
    
TestSet2: 
- Precision: 0.73
- Recall: 0.80

Если же принять, что выделение внутреннего квадрата это верное выделение (допустим, мы и там, и там сможем верно обнаружить центр этих квадратов), то результаты такие:

TestSet1:
- Precision: 1.00
- Recall: 0.87
    
TestSet2: 
- Precision: 1.00
- Recall: 0.83